## Import the dataset: `resized coco demo` from kaggle into the kaggle input directory. You can search `resized coco demo` in the datasets and you will find it.

In [ ]:
import subprocess
import time
import os 
from tqdm.auto import tqdm
import PIL

In [ ]:
# Cloning the github repo of EveryDream v2
!git clone https://github.com/victorchall/EveryDream2trainer.git

In [ ]:
%cd /kaggle/working/
!mkdir -p /kaggle/working/drive/MyDrive/everydreamlogs/ckpt
%cd /kaggle/working/EveryDream2trainer
!python utils/get_yamls.py
%cd /kaggle/working/
!mkdir input
%cd /kaggle/working/EveryDream2trainer

In [ ]:
MODEL_LOCATION = "https://huggingface.co/CompVis/stable-diffusion-v-1-4-original/resolve/main/sd-v1-4-full-ema.ckpt"

In [ ]:
# Downloading the model from hugging face hub
If_Ckpt = False
import os

download_path = ""
save_name = ""

if ".co" in MODEL_LOCATION or "https" in MODEL_LOCATION or "www" in MODEL_LOCATION:
    MODEL_URL = MODEL_LOCATION
    print("Downloading...")
    !wget $MODEL_LOCATION
    print("Download completed!")
    download_path = os.path.join(os.getcwd(), os.path.basename(MODEL_URL))
else:
    save_name = MODEL_LOCATION


In [ ]:
# Upgrade the libraries to compatible versions
!pip install --upgrade huggingface_hub diffusers

In [ ]:
# Convert the model to diffusers model
save_name = ""
if download_path.endswith(".ckpt") or MODEL_LOCATION.endswith(".ckpt"):
    MODEL_LOCATION = "/kaggle/working/EveryDream2trainer/sd-v1-4-full-ema.ckpt"
    If_Ckpt = True
    save_path = download_path
    img_size = 512
    upscale_attention = False
    prediction_type = "epsilon"
    inference_yaml = "v1-inference.yaml"
    if If_Ckpt:
        save_name = os.path.join("/kaggle/working/drive/MyDrive/everydreamlogs/ckpt/", save_name)
        print(save_name)
    !python utils/convert_original_stable_diffusion_to_diffusers.py --scheduler_type ddim \
    --original_config_file $inference_yaml \
    --image_size $img_size \
    --checkpoint_path $MODEL_LOCATION \
    --prediction_type $prediction_type \
    --upcast_attention \
    --dump_path $save_name

In [ ]:
%cd /kaggle/working/EveryDream2trainer

In [ ]:
Save_optimizer_state = False
Disconnect_after_training =  True
resume = False
Learning_Rate = 1e-6
Match_text_to_Unet = False
Text_lr =  5e-7
Schedule = "constant"
Text_lr_scheduler = "constant"
lr_warmup_steps = 0
lr_decay_steps = 0
Text_lr_warmup_steps = 0
Text_lr_decay_steps = 0
if Match_text_to_Unet:
  Text_lr = Learning_Rate
  Text_lr_scheduler = Schedule
  Text_lr_warmup_steps = lr_warmup_steps

import json
file_path = "/kaggle/working/EveryDream2trainer/optimizer.json"
with open(file_path, 'r') as file:
    data = json.load(file)

data['base']['lr'] = Learning_Rate
data['text_encoder_overrides']['lr'] = Text_lr 
data['base']['lr_scheduler'] = Schedule
data['text_encoder_overrides']['lr_scheduler'] = Text_lr_scheduler
data['base']['lr_warmup_steps'] = lr_warmup_steps
data['base']['lr_decay_steps'] = lr_decay_steps
data['text_encoder_overrides']['lr_warmup_steps'] = Text_lr_warmup_steps
data['text_encoder_overrides']['lr_decay_steps'] = Text_lr_decay_steps

# Save the updated JSON data back to the file
with open(file_path, 'w') as file:
    json.dump(data, file, indent=4)

In [ ]:
Gradient = "--gradient_checkpointing "
textencode = "--disable_textenc_training"
shuffle = "--shuffle_tags "
Drive = ""
DX = ""
validate = ""
wandb_settings = ""
model = "/kaggle/working/drive/MyDrive/everydreamlogs/ckpt"
Clip_skip = "1"
Batch_Size = "4"
Gradient_steps = "1"
Conditional_DropOut = "0.04"
dataset = "/kaggle/input/resized-coco-demo/Resized COCO demo/"
Picture_flip = "0"
Max_Epochs = "6"
Project_Name = "MyProject"
Resolution = "512"
Sample_File = "path_to_sample_file"
Steps_between_samples = "10"
Save_every_N_epoch = "6"
Training_Seed = "42"
zero_frequency_noise = "0.01"

In [ ]:
command = [
    "python", "train.py",
    "--resume_ckpt", model,
    Gradient.strip(),
    textencode,
    shuffle.strip(),
    Drive,
    DX.strip(),
    "--clip_skip", Clip_skip,
    "--batch_size", Batch_Size,
    "--grad_accum", Gradient_steps,
    "--cond_dropout", Conditional_DropOut,
    "--data_root", dataset,
    "--flip_p", Picture_flip,
    "--log_step", "25",
    "--max_epochs", Max_Epochs,
    "--project_name", Project_Name,
    "--resolution", Resolution,
    "--sample_prompts", Sample_File,
    "--sample_steps", Steps_between_samples,
    "--save_every_n_epochs", Save_every_N_epoch,
    "--seed", Training_Seed,
    "--zero_frequency_noise_ratio", zero_frequency_noise,
]
command = [arg for arg in command if arg.strip()]

In [ ]:
!pip install compel

In [ ]:
!pip install bitsandbytes

In [ ]:
import subprocess
subprocess.run(command)